## NoSQL Data Modelling with Apache Cassandra 

#### The CSV file titled **event_datafile_new.csv**, created using etl.ipynb contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [1]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster() #(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [2]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [3]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

#### Import Python packages 

In [4]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create Apache Cassandra tables to run the following queries given by the Analytics team
Note, we need to model the database tables based on the queries

#### Query 1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
query = "CREATE TABLE IF NOT EXISTS sessions"
query = query + "(sessionId int, itemInSession int, artist varchar, song varchar, length float, PRIMARY KEY(sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessions (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [12]:
query = "SELECT artist, song, length FROM sessions WHERE sessionId = 338 AND itemInSession = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print (e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2. Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [26]:
query = "CREATE TABLE IF NOT EXISTS session_songs"
query = query + "(userId int, sessionId int, itemInSession int, \
                  artist varchar, song varchar, \
                  firstName varchar, lastName varchar, \
                  PRIMARY KEY((userId, sessionId), itemInSession))"   # since we need to order by itemInSession, we use it as the only clustering column
try:
    session.execute(query)
except Exception as e:
    print(e)

In [28]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [30]:
query = "SELECT artist, song, firstName, lastName FROM session_songs WHERE userId = 10 AND sessionId = 182;"
try:
    rows = session.execute(query)
except Exception as e:
    print (e)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)   # use lowercase for column names otherise Cassandra driver not able to handle camel case

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3. Get every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'


In [31]:
query = "CREATE TABLE IF NOT EXISTS user_songs"
query = query + "(song varchar, userId int, firstName varchar, lastName varchar, \
                  PRIMARY KEY(song, userId))"   # since we need to order by itemInSession, we use it as the only clustering column
try:
    session.execute(query)
except Exception as e:
    print(e)

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (song, userId, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [34]:
query = "SELECT firstName, lastName FROM user_songs WHERE song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print (e)

for row in rows:
    print (row.firstname, row.lastname)   # use lowercase for column names otherise Cassandra driver not able to handle camel case

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [36]:
query = "DROP TABLE IF EXISTS sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS session_songs"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_songs"
try:
    session.execute(query)
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [37]:
session.shutdown()
cluster.shutdown()